In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras.datasets import mnist
import numpy as np
from PIL import Image
import argparse
import math

C:\Users\husey_000\Miniconda3\envs\capsule-gans\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def generator_model():
    model = Sequential()
    model.add(Dense(input_dim=100, output_dim=1024))
    model.add(Activation('tanh'))
    model.add(Dense(128*7*7))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Reshape((7, 7, 128), input_shape=(128*7*7,)))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(64, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(1, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    return model

In [3]:
def discriminator_model():
    model = Sequential()
    model.add(
            Conv2D(64, (5, 5),
            padding='same',
            input_shape=(28, 28, 1))
            )
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (5, 5)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [4]:
def generator_containing_discriminator(g, d):
    model = Sequential()
    model.add(g)
    d.trainable = False
    model.add(d)
    return model

In [5]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, 0]
    return image

In [6]:
def train(BATCH_SIZE):
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train[:, :, :, None]
    X_test = X_test[:, :, :, None]
    # X_train = X_train.reshape((X_train.shape, 1) + X_train.shape[1:])
    d = discriminator_model()
    g = generator_model()
    d_on_g = generator_containing_discriminator(g, d)
    d_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    g_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    d_on_g.compile(loss='binary_crossentropy', optimizer=g_optim)
    d.trainable = True
    d.compile(loss='binary_crossentropy', optimizer=d_optim)
    for epoch in range(100):
        print("Epoch is", epoch)
        print("Number of batches", int(X_train.shape[0]/BATCH_SIZE))
        for index in range(int(X_train.shape[0]/BATCH_SIZE)):
            noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images = g.predict(noise, verbose=0)
            if index % 20 == 0:
                image = combine_images(generated_images)
                image = image*127.5+127.5
                Image.fromarray(image.astype(np.uint8)).save(
                    str(epoch)+"_"+str(index)+".png")
            X = np.concatenate((image_batch, generated_images))
            y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
            d_loss = d.train_on_batch(X, y)
            print("batch %d d_loss : %f" % (index, d_loss))
            noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
            d.trainable = False
            g_loss = d_on_g.train_on_batch(noise, [1] * BATCH_SIZE)
            d.trainable = True
            print("batch %d g_loss : %f" % (index, g_loss))
            if index % 10 == 9:
                g.save_weights('generator', True)
                d.save_weights('discriminator', True)

In [7]:
def generate(BATCH_SIZE, nice=False):
    g = generator_model()
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    g.load_weights('generator')
    if nice:
        d = discriminator_model()
        d.compile(loss='binary_crossentropy', optimizer="SGD")
        d.load_weights('discriminator')
        noise = np.random.uniform(-1, 1, (BATCH_SIZE*20, 100))
        generated_images = g.predict(noise, verbose=1)
        d_pret = d.predict(generated_images, verbose=1)
        index = np.arange(0, BATCH_SIZE*20)
        index.resize((BATCH_SIZE*20, 1))
        pre_with_index = list(np.append(d_pret, index, axis=1))
        pre_with_index.sort(key=lambda x: x[0], reverse=True)
        nice_images = np.zeros((BATCH_SIZE,) + generated_images.shape[1:3], dtype=np.float32)
        nice_images = nice_images[:, :, :, None]
        for i in range(BATCH_SIZE):
            idx = int(pre_with_index[i][1])
            nice_images[i, :, :, 0] = generated_images[idx, :, :, 0]
        image = combine_images(nice_images)
    else:
        noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
        generated_images = g.predict(noise, verbose=1)
        image = combine_images(generated_images)
    image = image*127.5+127.5
    Image.fromarray(image.astype(np.uint8)).save(
        "generated_image.png")

In [8]:
train(BATCH_SIZE=32)

C:\Users\husey_000\Miniconda3\envs\capsule-gans\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1024, input_dim=100)`
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch is 0
Number of batches 1875
batch 0 d_loss : 0.686795
batch 0 g_loss : 0.668823
batch 1 d_loss : 0.679754
batch 1 g_loss : 0.678705
batch 2 d_loss : 0.664826
batch 2 g_loss : 0.661664
batch 3 d_loss : 0.654493
batch 3 g_loss : 0.646359
batch 4 d_loss : 0.638615
batch 4 g_loss : 0.642812
batch 5 d_loss : 0.630914
batch 5 g_loss : 0.632877
batch 6 d_loss : 0.610103
batch 6 g_loss : 0.619243
batch 7 d_loss : 0.590622
batch 7 g_loss : 0.612112
batch 8 d_loss : 0.570343
batch 8 g_loss : 0.611955
batch 9 d_loss : 0.556252
batch 9 g_loss : 0.590085
batch 10 d_loss : 0.547110
batch 10 g_loss : 0.585732
batch 11 d_loss : 0.533157
batch 11 g_loss : 0.573667
batch 12 d_loss : 0.513140
batch 12 g_loss : 0.564141
batch 13 d_loss : 0.509552
batch 13 g_loss : 0.555272
batch 14 d_loss : 0.503973
batch 14 g_loss : 0.545077
batch 15 d_loss : 0.491388
batch 15 g_loss : 0.534188
batch 16 d_loss : 0.483716
batch 16 g_loss : 0.534831
batch 17 d_loss : 0.479313
batch 17 g_loss : 0.530424
batch 18 d_los

batch 150 d_loss : 0.214923
batch 150 g_loss : 0.545337
batch 151 d_loss : 0.233073
batch 151 g_loss : 0.522663
batch 152 d_loss : 0.226032
batch 152 g_loss : 0.541583
batch 153 d_loss : 0.263941
batch 153 g_loss : 0.561126
batch 154 d_loss : 0.241218
batch 154 g_loss : 0.545695
batch 155 d_loss : 0.235945
batch 155 g_loss : 0.504086
batch 156 d_loss : 0.245093
batch 156 g_loss : 0.527991
batch 157 d_loss : 0.215472
batch 157 g_loss : 0.520247
batch 158 d_loss : 0.244664
batch 158 g_loss : 0.559696
batch 159 d_loss : 0.218886
batch 159 g_loss : 0.541788
batch 160 d_loss : 0.230111
batch 160 g_loss : 0.542152
batch 161 d_loss : 0.220244
batch 161 g_loss : 0.519909
batch 162 d_loss : 0.239166
batch 162 g_loss : 0.531191
batch 163 d_loss : 0.224933
batch 163 g_loss : 0.561390
batch 164 d_loss : 0.214040
batch 164 g_loss : 0.540813
batch 165 d_loss : 0.247690
batch 165 g_loss : 0.586207
batch 166 d_loss : 0.213660
batch 166 g_loss : 0.584054
batch 167 d_loss : 0.208941
batch 167 g_loss : 0

batch 299 d_loss : 0.113195
batch 299 g_loss : 0.208448
batch 300 d_loss : 0.088048
batch 300 g_loss : 0.221407
batch 301 d_loss : 0.104301
batch 301 g_loss : 0.231304
batch 302 d_loss : 0.087715
batch 302 g_loss : 0.268482
batch 303 d_loss : 0.088600
batch 303 g_loss : 0.268482
batch 304 d_loss : 0.085681
batch 304 g_loss : 0.174289
batch 305 d_loss : 0.065058
batch 305 g_loss : 0.199071
batch 306 d_loss : 0.117871
batch 306 g_loss : 0.239663
batch 307 d_loss : 0.070387
batch 307 g_loss : 0.236458
batch 308 d_loss : 0.110190
batch 308 g_loss : 0.179563
batch 309 d_loss : 0.127447
batch 309 g_loss : 0.171891
batch 310 d_loss : 0.116028
batch 310 g_loss : 0.256839
batch 311 d_loss : 0.083236
batch 311 g_loss : 0.195501
batch 312 d_loss : 0.101771
batch 312 g_loss : 0.173553
batch 313 d_loss : 0.074910
batch 313 g_loss : 0.135101
batch 314 d_loss : 0.109469
batch 314 g_loss : 0.175374
batch 315 d_loss : 0.096908
batch 315 g_loss : 0.168273
batch 316 d_loss : 0.105079
batch 316 g_loss : 0

batch 445 g_loss : 0.089749
batch 446 d_loss : 0.177419
batch 446 g_loss : 0.086594
batch 447 d_loss : 0.198484
batch 447 g_loss : 0.114443
batch 448 d_loss : 0.207304
batch 448 g_loss : 0.129404
batch 449 d_loss : 0.221461
batch 449 g_loss : 0.088106
batch 450 d_loss : 0.202005
batch 450 g_loss : 0.086627
batch 451 d_loss : 0.235573
batch 451 g_loss : 0.104576
batch 452 d_loss : 0.199266
batch 452 g_loss : 0.128547
batch 453 d_loss : 0.302198
batch 453 g_loss : 0.109738
batch 454 d_loss : 0.228777
batch 454 g_loss : 0.077604
batch 455 d_loss : 0.217379
batch 455 g_loss : 0.079173
batch 456 d_loss : 0.269206
batch 456 g_loss : 0.093355
batch 457 d_loss : 0.236525
batch 457 g_loss : 0.081023
batch 458 d_loss : 0.214786
batch 458 g_loss : 0.097699
batch 459 d_loss : 0.230321
batch 459 g_loss : 0.164803
batch 460 d_loss : 0.266544
batch 460 g_loss : 0.095016
batch 461 d_loss : 0.295005
batch 461 g_loss : 0.113866
batch 462 d_loss : 0.300929
batch 462 g_loss : 0.133345
batch 463 d_loss : 0

...

batch 1463 d_loss : 0.032475
batch 1463 g_loss : 0.018579
batch 1464 d_loss : 0.051023
batch 1464 g_loss : 0.007268
batch 1465 d_loss : 0.035172
batch 1465 g_loss : 0.009096
batch 1466 d_loss : 0.082026
batch 1466 g_loss : 0.006704
batch 1467 d_loss : 0.029543
batch 1467 g_loss : 0.011236
batch 1468 d_loss : 0.043196
batch 1468 g_loss : 0.009634
batch 1469 d_loss : 0.057628
batch 1469 g_loss : 0.010488
batch 1470 d_loss : 0.038994
batch 1470 g_loss : 0.007976
batch 1471 d_loss : 0.061260
batch 1471 g_loss : 0.013451
batch 1472 d_loss : 0.048596
batch 1472 g_loss : 0.008619
batch 1473 d_loss : 0.063635
batch 1473 g_loss : 0.006706
batch 1474 d_loss : 0.045372
batch 1474 g_loss : 0.010281
batch 1475 d_loss : 0.063020
batch 1475 g_loss : 0.007591
batch 1476 d_loss : 0.049283
batch 1476 g_loss : 0.005906
batch 1477 d_loss : 0.083213
batch 1477 g_loss : 0.012552
batch 1478 d_loss : 0.033232
batch 1478 g_loss : 0.004832
batch 1479 d_loss : 0.052239
batch 1479 g_loss : 0.018714
batch 1480 d_l

batch 1606 d_loss : 0.044681
batch 1606 g_loss : 0.012949
batch 1607 d_loss : 0.083429
batch 1607 g_loss : 0.005983
batch 1608 d_loss : 0.073674
batch 1608 g_loss : 0.003945
batch 1609 d_loss : 0.043085
batch 1609 g_loss : 0.004852
batch 1610 d_loss : 0.073453
batch 1610 g_loss : 0.003757
batch 1611 d_loss : 0.047844
batch 1611 g_loss : 0.005068
batch 1612 d_loss : 0.043877
batch 1612 g_loss : 0.005393
batch 1613 d_loss : 0.052446
batch 1613 g_loss : 0.003671
batch 1614 d_loss : 0.083191
batch 1614 g_loss : 0.003232
batch 1615 d_loss : 0.027748
batch 1615 g_loss : 0.003181
batch 1616 d_loss : 0.132154
batch 1616 g_loss : 0.003133
batch 1617 d_loss : 0.059428
batch 1617 g_loss : 0.008042
batch 1618 d_loss : 0.076077
batch 1618 g_loss : 0.009573
batch 1619 d_loss : 0.045103
batch 1619 g_loss : 0.011147
batch 1620 d_loss : 0.039311
batch 1620 g_loss : 0.021317
batch 1621 d_loss : 0.029488
batch 1621 g_loss : 0.016535
batch 1622 d_loss : 0.027140
batch 1622 g_loss : 0.007766
batch 1623 d_l

batch 1748 g_loss : 0.002769
batch 1749 d_loss : 0.037023
batch 1749 g_loss : 0.005872
batch 1750 d_loss : 0.065598
batch 1750 g_loss : 0.003806
batch 1751 d_loss : 0.040084
batch 1751 g_loss : 0.006041
batch 1752 d_loss : 0.055086
batch 1752 g_loss : 0.004313
batch 1753 d_loss : 0.067079
batch 1753 g_loss : 0.006359
batch 1754 d_loss : 0.027119
batch 1754 g_loss : 0.007594
batch 1755 d_loss : 0.085709
batch 1755 g_loss : 0.007547
batch 1756 d_loss : 0.035859
batch 1756 g_loss : 0.002407
batch 1757 d_loss : 0.031892
batch 1757 g_loss : 0.004492
batch 1758 d_loss : 0.067746
batch 1758 g_loss : 0.006693
batch 1759 d_loss : 0.046235
batch 1759 g_loss : 0.006190
batch 1760 d_loss : 0.081139
batch 1760 g_loss : 0.003232
batch 1761 d_loss : 0.046627
batch 1761 g_loss : 0.003074
batch 1762 d_loss : 0.030316
batch 1762 g_loss : 0.003955
batch 1763 d_loss : 0.049474
batch 1763 g_loss : 0.011615
batch 1764 d_loss : 0.042626
batch 1764 g_loss : 0.003971
batch 1765 d_loss : 0.109693
batch 1765 g_l

batch 16 g_loss : 0.001254
batch 17 d_loss : 0.040103
batch 17 g_loss : 0.001183
batch 18 d_loss : 0.045404
batch 18 g_loss : 0.003571
batch 19 d_loss : 0.051388
batch 19 g_loss : 0.002041
batch 20 d_loss : 0.042059
batch 20 g_loss : 0.003103
batch 21 d_loss : 0.033572
batch 21 g_loss : 0.005851
batch 22 d_loss : 0.030707
batch 22 g_loss : 0.002617
batch 23 d_loss : 0.043114
batch 23 g_loss : 0.001666
batch 24 d_loss : 0.069578
batch 24 g_loss : 0.002507
batch 25 d_loss : 0.038722
batch 25 g_loss : 0.002987
batch 26 d_loss : 0.068602
batch 26 g_loss : 0.001085
batch 27 d_loss : 0.038343
batch 27 g_loss : 0.002638
batch 28 d_loss : 0.040617
batch 28 g_loss : 0.007858
batch 29 d_loss : 0.030253
batch 29 g_loss : 0.002755
batch 30 d_loss : 0.038934
batch 30 g_loss : 0.005009
batch 31 d_loss : 0.058725
batch 31 g_loss : 0.001268
batch 32 d_loss : 0.048802
batch 32 g_loss : 0.001942
batch 33 d_loss : 0.125994
batch 33 g_loss : 0.001060
batch 34 d_loss : 0.025777
batch 34 g_loss : 0.003645
b

KeyboardInterrupt: 

In [9]:
generate(BATCH_SIZE=32, nice=False)

C:\Users\husey_000\Miniconda3\envs\capsule-gans\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1024, input_dim=100)`
  This is separate from the ipykernel package so we can avoid doing imports until


32/32 [==============================] - 0s 2ms/step
